1. *Initialize* the weights.
1. For each image, use these weights to *predict* whether it appears to be a 3 or a 7.
1. Based on these predictions, calculate how good the model is (its *loss*).
1. Calculate the *gradient*, which measures for each weight, how changing that weight would change the loss
1. *Step* (that is, change) all the weights based on that calculation.
1. Go back to the step 2, and *repeat* the process.
1. Iterate until you decide to *stop* the training process (for instance, because the model is good enough or you don't want to wait any longer).

In [ ]:
def apply_step(params, prn=True):
    preds = f(time, params)
    loss = mse(preds, speed)
    loss.backward()
    params.data -= lr * params.grad.data
    params.grad = None
    if prn: print(loss.item())
    return preds

### Primero importemos las librerias de fast ia

In [1]:
import fastbook
fastbook.setup_book()

In [2]:
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [3]:
import pandas as pd
import torch as th

# Ahora importemos el data set

In [4]:
path = untar_data(URLs.MNIST_SAMPLE)
print(path)

/storage/data/mnist_sample


Vemos que path posee una muestra de minist

In [5]:
path.ls()

(#3) [Path('/storage/data/mnist_sample/train'),Path('/storage/data/mnist_sample/labels.csv'),Path('/storage/data/mnist_sample/valid')]

Vemos que tenemos el conjunto de validacion, de entrenamiento y un csv con las etiquetas

In [6]:
def tensorImage (ruta): # recibo una ruta con una serie de imagenes convertir en tensor
    return torch.stack([tensor(Image.open(i)) for i in ruta]).float()/255

In [7]:
inPuts = torch.cat([ tensorImage(((path/'train/3').ls())), tensorImage(((path/'train/7').ls())) ]).view([-1, 28*28])
targets = tensor([0]*len((path/'train/3').ls())+ [1]* len((path/'train/7').ls())).unsqueeze(1)

In [8]:
validSet = torch.cat([ tensorImage(((path/'valid/3').ls())), tensorImage(((path/'valid/7').ls())) ]).view([-1, 28*28])
validTargets = tensor([0]*len((path/'train/3').ls())+ [1]* len((path/'train/7').ls())).unsqueeze(1)

# Data loader

Ahora que tenemos un un conjunto de datos, podemos hacer un data loader con ayuda de pytorch

In [12]:
class MyDataloader:

    def __init__(self, trainSet, trainTargets, validSet, validTargets, batchSize):    
        #guardo los data sets emparejados con sus respectivas etiquetas de salida
        self.batchSize = batchSize
        self.trainSet = self.zipSet(trainSet, trainTargets)
        self.validSet = self.zipSet(validSet, validTargets)
        self.trainBatches = MyDataloader.genBatches(self.trainSet, batchSize)
        self.validBatches = MyDataloader.genBatches(self.validSet, batchSize)
    def zipSet (self, inPut, target):
        #Retorna una lista de duplas a partir un tensor de inputs y un tensor de targets asociados a dicho input
        duplas= zip(inPut, target)
        return list(duplas)
    
    def genBatches ( Dset, batchSize):
        #recibe una lista y un tamaño de lote, para retornar una lista con sub listas de la lista recibida del tamaño indicdo
        batches = []                   #armo una lista vacia donde guardar los distintos lotes
        nElements = len(Dset)          #miro cuantos elementos tengo que randomizar
        index = th.randperm(nElements)      #genero una lista de indices randomizados
        nBatches = int(nElements/batchSize)
        for i in range (nBatches):
            batchAux = Dset[batchSize*i : batchSize*(i+1)]
            batches.append(batchAux)
        #la conversion a int nos dejo fuera de la iteracion n elementos que son el resto de la divicion para nBatches
        if(nElements % batchSize != 0): #si mi divicion de lotes no es exacta
            batches.append(Dset[batchSize*(i+1) : -1]) # agrego un ultimo lote con los elementos faltantes
        return batches
    
    #para acceso a los batches = [numero de batch] [numero de dupla input/target] [elemento de la dupla ]

In [22]:
dl = MyDataloader(inPuts, targets, validSet, validTargets, 1000)

            

# Modelo

El modelo de nuestra red, basicamente se reduce a los parametros/pesos (weights) de las neuronas, a los baias y a las forma de calcular lass predicciones, osea como usar los pesos y los baias. Implementemos un modelo de 1 capa que reciba un una cantidad de neuronas para la capa y posea un metodo para predecir con dichas neuronas

In [23]:
class Modelo:
    
    def __init__(self, nNeuronas):
        self.wights = torch.randn(nNeuronas).requires_grad_()
        self.bias = torch.randn(1).requires_grad_()
    
    def predict(self, inPuts):
        return th.sigmoid((inPuts @ self.wights).sum() + self.bias)

In [24]:
modelo = Modelo(28*28)

In [27]:
modelo.predict(dl.trainBatches[0][0][0])

tensor([0.6518], grad_fn=<SigmoidBackward>)

# Ahora obtengamos el loss

Ahora que tenemos nuestro modelo, necesitamos una forma de medir que tan bien o mal esta nuestr prediccion, el loss, la cual es simplemente una funcion.

In [56]:
def mnist_loss(predictions, targets):
    return torch.where(targets==1, 1-predictions, predictions).mean()

# Optimizer

El optimizer es el encargado de realizar la actualizacion de los parametros del modelo, mediante la backpropagation generada a partir del aclculo del loss. por lo que sera una funcion que necesitara de los parametros del modelo y el resultado del loss.

In [55]:
def optimizer (params, bias, loss, lr):
    loss.backward()           #calcula el gradiente a partir de loss
    params.data = params.data - params.grad * lr #actualizar params y bias
    bias.data = bias.data - bias.grad * lr
    params.grad.zero_() # restablece los gradientes
    bias.grad.zero_()

# Definamos la metrica (error cuadratico medio)

In [ ]:
def errCuad (pred, target):
    return ((pred - target)**2).mean(-1,-2).sqrt()

# Learner

Un learner debe de poder contener, un conjunto de datos, los parametros del modelo, la metrica, el loss y el optimizer. Y luego debe brindar la funcionalidad de entrenar el modelo.